<a href="https://colab.research.google.com/github/DSWoodin/Pythonworkspace/blob/main/ProDS/%EB%8B%A4%EC%A4%91%ED%9A%8C%EA%B7%80%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install patsy

In [3]:
import pandas as pd
from patsy import dmatrices
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif

In [5]:
df = pd.read_csv('bike.csv')
df.head(2)

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40


In [6]:
df_sub = df.loc[:, 'season':'casual']
df_sub.head(2)

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual
0,1,0,0,1,9.84,14.395,81,0.0,3
1,1,0,0,1,9.02,13.635,80,0.0,8


In [7]:
'causual ~ season + holiday + ... + windspeed'

'causual ~ season + holiday + ... + windspeed'

In [8]:
'casual ~ ' + 'season + holiday'

'casual ~ season + holiday'

In [9]:
'casual ~ ' + ' + '.join(df_sub.columns)

'casual ~ season + holiday + workingday + weather + temp + atemp + humidity + windspeed + casual'

In [10]:
'casual ~ ' + ' + '.join(df_sub.columns[:-1])

'casual ~ season + holiday + workingday + weather + temp + atemp + humidity + windspeed'

In [11]:
formula = 'casual ~ ' + ' + '.join(df_sub.columns[:-1])
y, X = dmatrices(formula, data = df_sub, return_type='dataframe')

In [12]:
y.head(2)

,casual
0,3.0
1,8.0


In [13]:
X.head(2)

,Intercept,season,holiday,workingday,weather,temp,atemp,humidity,windspeed
0,1.0,1.0,0.0,0.0,1.0,9.84,14.395,81.0,0.0
1,1.0,1.0,0.0,0.0,1.0,9.02,13.635,80.0,0.0


In [14]:
df_vif = pd.DataFrame()
df_vif['colname'] = X.columns
df_vif['VIF'] = [vif(X.values, i) for i in range(X.shape[1])]
df_vif

,colname,VIF
0,Intercept,34.029472
1,season,1.137211
2,holiday,1.069731
3,workingday,1.071196
4,weather,1.236150
5,temp,35.516012
6,atemp,35.550831
7,humidity,1.425034
8,windspeed,1.195704


In [15]:
df_sub = pd.concat([df.loc[:, 'season':'temp'],
                   df.loc[:, 'humidity':'casual']],
                   axis=1)
df_sub.head(2)

,season,holiday,workingday,weather,temp,humidity,windspeed,casual
0,1,0,0,1,9.84,81,0.0,3
1,1,0,0,1,9.02,80,0.0,8


In [16]:
formula = 'casual ~ ' + ' + '.join(df_sub.columns[:-1])
y, X = dmatrices(formula, data = df_sub, return_type='dataframe')

df_vif = pd.DataFrame()
df_vif['colname'] = X.columns
df_vif['VIF'] = [vif(X.values, i) for i in range(X.shape[1])]
df_vif

# atemp를 제거하니 temp의 VIF가 35에서 1로 줄어듬
# 다중공선성의 문제가 해결되었음

,colname,VIF
0,Intercept,31.375118
1,season,1.136866
2,holiday,1.068094
3,workingday,1.070025
4,weather,1.235251
5,temp,1.089028
6,humidity,1.421256
7,windspeed,1.149650


In [17]:
df = df.iloc[:, 1:]
df.corr().round(2)

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
season,1.00,0.03,-0.01,0.01,0.26,0.26,0.19,-0.15,0.10,0.16,0.16
holiday,0.03,1.00,-0.25,-0.01,0.00,-0.01,0.00,0.01,0.04,-0.02,-0.01
workingday,-0.01,-0.25,1.00,0.03,0.03,0.02,-0.01,0.01,-0.32,0.12,0.01
weather,0.01,-0.01,0.03,1.00,-0.06,-0.06,0.41,0.01,-0.14,-0.11,-0.13
temp,0.26,0.00,0.03,-0.06,1.00,0.98,-0.06,-0.02,0.47,0.32,0.39
atemp,0.26,-0.01,0.02,-0.06,0.98,1.00,-0.04,-0.06,0.46,0.31,0.39
humidity,0.19,0.00,-0.01,0.41,-0.06,-0.04,1.00,-0.32,-0.35,-0.27,-0.32
windspeed,-0.15,0.01,0.01,0.01,-0.02,-0.06,-0.32,1.00,0.09,0.09,0.10
casual,0.10,0.04,-0.32,-0.14,0.47,0.46,-0.35,0.09,1.00,0.50,0.69
registered,0.16,-0.02,0.12,-0.11,0.32,0.31,-0.27,0.09,0.50,1.00,0.97


In [18]:
df.head(2)

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,1,0,0,1,9.02,13.635,80,0.0,8,32,40


In [20]:
df_dum = pd.get_dummies(df, columns=['season'], drop_first=True)
df_dum.head(2)

,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,season_2,season_3,season_4
0,0,0,1,9.84,14.395,81,0.0,3,13,16,False,False,False
1,0,0,1,9.02,13.635,80,0.0,8,32,40,False,False,False


### 문제 1: Price를 종속변수로 하고 나머지 수치형 변수를 독립변수로 했을 때 다중 공선성의 문제가 있다고 판단되는 변수의 개수는?

In [21]:
df = pd.read_csv('diamonds.csv')
df.head(2)

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31


In [22]:
df_sub = df.iloc[:, [6,0,4,5,7,8,9]]
df_sub.head(2)

,price,carat,depth,table,x,y,z
0,326,0.23,61.5,55.0,3.95,3.98,2.43
1,326,0.21,59.8,61.0,3.89,3.84,2.31


In [23]:
y, X = dmatrices('price ~ ' + ' + '.join(df_sub.columns[1:]),
                 data = df_sub, return_type='dataframe')
df_vif = pd.DataFrame()
df_vif['vars'] = X.columns
df_vif['VIF'] = [vif(X.values, i) for i in range(X.shape[1])]
df_vif

,vars,VIF
0,Intercept,4821.696350
1,carat,21.602712
2,depth,1.496590
3,table,1.143225
4,x,56.187704
5,y,20.454295
6,z,23.530049


### 문제 2: price를 종속변수로 하고 carat과 depth를 독립변수로 하여 생성한 선형 회귀 모델을 사용하여 알아본 carat이 1이고, depth가 60, table이 55인 다이아몬드의 가격은 얼마인가?

In [24]:
df = pd.read_csv('diamonds.csv')
df.head(2)

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31


In [25]:
from statsmodels.formula.api import ols

In [27]:
model = ols(formula = 'price ~ carat + depth', data=df).fit()
[model.summary()]

[<class 'statsmodels.iolib.summary.Summary'>
 """
                             OLS Regression Results                            
 Dep. Variable:                  price   R-squared:                       0.851
 Model:                            OLS   Adj. R-squared:                  0.851
 Method:                 Least Squares   F-statistic:                 1.536e+05
 Date:                Mon, 29 Jul 2024   Prob (F-statistic):               0.00
 Time:                        05:25:25   Log-Likelihood:            -4.7249e+05
 No. Observations:               53940   AIC:                         9.450e+05
 Df Residuals:                   53937   BIC:                         9.450e+05
 Df Model:                           2                                         
 Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
 ---------------------------------------------------------------------

In [28]:
df_test = pd.DataFrame({'carat':[1], 'depth':[60], 'table':[55]})
df_test

,carat,depth,table
0,1,60,55


In [29]:
model.predict(df_test)

0    5680.554517
dtype: float64

### 문제 3: price를 종속변수로 하고 carat, color, depth를 독립변수로 하여 생성한 선형 회귀 모델을 사용하여 알아본 carat이 1이고, depth가 50, color가 E인 다이아몬드의 가격은 얼마인가?

In [30]:
df = pd.read_csv('diamonds.csv')
df.head(2)

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31


In [31]:
df_sub = df.loc[:, ['price', 'carat', 'color', 'depth']]
df_sub.head(2)

,price,carat,color,depth
0,326,0.23,E,61.5
1,326,0.21,E,59.8


In [32]:
df_dum = pd.get_dummies(df_sub, columns=['color'], drop_first=True)
df_dum.head(2)

,price,carat,depth,color_E,color_F,color_G,color_H,color_I,color_J
0,326,0.23,61.5,True,False,False,False,False,False
1,326,0.21,59.8,True,False,False,False,False,False


In [33]:
model = ols(formula = 'price ~ ' + ' + '.join(df_dum.columns[1:]), data=df_dum).fit()
[model.summary()]

[<class 'statsmodels.iolib.summary.Summary'>
 """
                             OLS Regression Results                            
 Dep. Variable:                  price   R-squared:                       0.865
 Model:                            OLS   Adj. R-squared:                  0.865
 Method:                 Least Squares   F-statistic:                 4.319e+04
 Date:                Mon, 29 Jul 2024   Prob (F-statistic):               0.00
 Time:                        05:29:26   Log-Likelihood:            -4.6977e+05
 No. Observations:               53940   AIC:                         9.396e+05
 Df Residuals:                   53931   BIC:                         9.396e+05
 Df Model:                           8                                         
 Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
 ----------------------------------------------------------------

In [34]:
df_test = df_dum.iloc[[0], ]
df_test

,price,carat,depth,color_E,color_F,color_G,color_H,color_I,color_J
0,326,0.23,61.5,True,False,False,False,False,False


In [35]:
df_test['carat'] = 1
df_test['depth'] = 50
df_test

<ipython-input-35-bdedfaba7d17>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['carat'] = 1
<ipython-input-35-bdedfaba7d17>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['depth'] = 50


,price,carat,depth,color_E,color_F,color_G,color_H,color_I,color_J
0,326,1,50,True,False,False,False,False,False


In [36]:
model.predict(df_test)

0    6884.782287
dtype: float64